In [9]:
!pip install -U -q selenium
!pip install -q webdriver-manager
!pip install pymysql

In [3]:
## Import required libraries
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from time import sleep
import json

import pandas as pd
import urllib.parse
from tqdm import tqdm, notebook
from datetime import date
import datetime
from tqdm import tqdm
import pickle
import re

# Yocket Login Information

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://yocket.com/login")
time.sleep(5)

In [3]:
driver.find_element(By.CSS_SELECTOR,"span.inline-block.text-yocket-orange-700.text-sm.font-bold.cursor-pointer.mt-4").click()
time.sleep(1)

In [6]:
key=input("Login Credentials: ") ##login Credentials
email = driver.find_element(By.CSS_SELECTOR,"input.text-left.lowercase.rounded-lg.mt-2.w-full.focus\:ring-yocket-orange-700.focus\:border-neutral-300.border-neutral-300")
email.send_keys(key)
time.sleep(1)
driver.find_element(By.CSS_SELECTOR,"button.md\:h-10.line-clamp-1.rounded-lg.py-3.md\:py-2.px-2.text-base.md\:text-sm.font-bold.w-full.text-center.bg-yocket-orange-700.customShadow.text-white").click()

### Note: Provide OTP Information Manually

# Yocket High Level USA Aspirants for Fall Term

In [5]:
todays_date = date.today()
current_year = todays_date.year

todays_date = date.today()
current_year = todays_date.year + 2 
years_list = []
for i in range(1,6): ## Current and Last 4 Admission Years 
    years_list.append(str(current_year - i))

total_aspirants = {}
for i in tqdm(years_list):
    yearly_aspirants = {}
    driver.get("https://yocket.com/connect?country=United%20States&term=Fall&year={}".format(i))
    time.sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    total_value = soup.find('p',"text-gray-700").text.strip()
    total_aspirants[i] = total_value

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:04<00:00, 12.84s/it]


In [56]:
df_usa_aspirants = pd.DataFrame.from_dict(total_aspirants.items())
df_usa_aspirants.rename(columns={0: 'year', 1:'student_count_text'},inplace=True)
df_usa_aspirants['student_count'] = df_usa_aspirants['student_count_text'].str[19:].str.split('aspirant').str[0].astype(int)
df_usa_aspirants['year'] = df_usa_aspirants['year'].astype(int)

# Yocket MSBA High Level USA Aspirants for Fall Term

In [62]:
total_aspirants_ba = {}
for i in years_list:
    driver.get("https://yocket.com/connect?course=Business%20Analytics&country=United%20States&term=Fall&year={}".format(i))
    time.sleep(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    total_value = soup.find('p',"text-gray-700").text.strip()
    total_aspirants_ba[i] = total_value

In [63]:
df_msba_aspirants = pd.DataFrame.from_dict(total_aspirants_ba.items())
df_msba_aspirants.rename(columns={0: 'year', 1:'student_count_text'},inplace=True)
df_msba_aspirants['student_count'] = df_msba_aspirants['student_count_text'].str[19:].str.split('aspirant').str[0].astype(int)
df_msba_aspirants['year'] = df_msba_aspirants['year'].astype(int)

# Yocket Competitor Numbers

In [67]:
#Check Univs mapping for confirmation
school_id_dict = {}
school_id_dict[868]  = 'University of California, Davis'
school_id_dict[713]  = 'University of Texas, Austin (McCombs)'
school_id_dict[707]  = 'University of California, Los Angeles (Anderson)'
school_id_dict[745]  = 'University of Southern California (Marshall)'
school_id_dict[1480] = 'University of Washington (Foster)'
school_id_dict[702]  = 'Carnegie Mellon University (Tepper)'
school_id_dict[748]  = 'Purdue University, West Lafayette (Krannert)'
school_id_dict[709]  = 'Columbia University'
school_id_dict[764]  = 'University of California, San Diego'
school_id_dict[1094] = 'University of Minnesota, Twin Cities (Carlson)'
school_id_dict[2833] = 'University of San Francisco (Masagung)'
school_id_dict[743]  = 'University of Cincinnati'
school_id_dict[711]  = 'University of Illinois--Urbana-Champaign'

In [82]:
school_year_wise_aspirants = []
for i in tqdm(range(len(years_list))):
    for uni in school_id_dict.keys():
        applicant = {}
        applicant['year'] = years_list[i]
        applicant['university'] = school_id_dict[uni]
        driver.get("https://yocket.com/connect?university={}&course=Business%20Analytics&country=United%20States&term=Fall&year={}".format(uni,years_list[i]))
        time.sleep(10)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        value = soup.find('p',"text-gray-700").text.strip()
        applicant['student_count_text'] = value
        school_year_wise_aspirants.append(applicant)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [14:11<00:00, 170.33s/it]


In [83]:
df_school_year = pd.DataFrame(school_year_wise_aspirants)
df_school_year['student_count'] = df_school_year['student_count_text'].str[19:].str.split('aspirant').str[0].astype(int)
df_school_year['year'] = df_school_year['year'].astype(int)

## Yocket Current Year Students

In [93]:
## Collect the ids of the Students aiming for the current term
## if the process stops in between continue based on the number of extractions completed 
## It usually takes longer to extract the ids
## It can be improved if possible 
## Same can be used to extract past year data as well with just changing the year 
## University filter can also be added in the url to get specific univs aspirants

id_list=[]
driver.get("https://yocket.com/connect?course=Business%20Analytics&country=United%20States&term=Fall&year={}".format(int(years_list[0])))
time.sleep(15)
total = int(driver.find_element(By.CSS_SELECTOR,"p.text-gray-700").text.split("aspirants")[0].replace("Showing 9 of","").strip())
i = 0
file_path = 'abcd.pkl' ## Mention the filepath where it needs to be saved
while i < total:
    rept = int(driver.find_element(By.CSS_SELECTOR,"p.text-gray-700").text.split("Showing")[1].strip()[0])
    while i >= rept:
        if i >= rept:
            driver.find_element(By.CSS_SELECTOR,"button.mx-auto.w-80.bg-yocket-neutral-900.text-white.text-center.rounded-lg.py-2").click()
            time.sleep(10)
            rept = int(driver.find_element(By.CSS_SELECTOR,"p.text-gray-700").text.split("Showing")[1].strip()[0])
            break
    elements = driver.find_elements(By.CSS_SELECTOR,"div.bg-white.border.rounded-xl.w-full.flex.flex-col.pb-6.mb-2.cursor-pointer.transition.ease-in-out.md\:hover\:-translate-y-1.md\:hover\:scale-105.duration-300.hover\:shadow-lg")
    if i%20 ==0:
        with open(file_path, 'wb') as handle:
            pickle.dump(id_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    for element in elements:
        try:
            if element.get_attribute("data-se") == ('AspirantCard-'+str(i)):
                element.click()
                time.sleep(10)
                id_list.append(driver.find_element(By.CSS_SELECTOR,"p.mt-1.text-yocket-neutral-500.text-center").text)
                i+=1
                print(i)
                driver.get("https://yocket.com/connect?course=Business%20Analytics&country=United%20States&term=Fall&year={}".format(int(years_list[0])))
                time.sleep(10)
                break
        except:
            print("error occurred: going back to main page")
            driver.get("https://yocket.com/connect?course=Business%20Analytics&country=United%20States&term=Fall&year={}".format(int(years_list[0])))
            time.sleep(15)
            while i >= rept:
                if i >= rept:
                    driver.find_element(By.CSS_SELECTOR,"button.mx-auto.w-80.bg-yocket-neutral-900.text-white.text-center.rounded-lg.py-2").click()
                    time.sleep(5)
    if i == total:
        break

## Yocket Student Profiles

In [95]:
for i in tqdm(id_list):
    driver.get("https://yocket.com/profile/{}".format(i[1:]))
    time.sleep(15)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    profile_dictionary = {}
    profile_dictionary['id'] = i
    print(id_list.index(i))
    for item in soup.find_all("div",class_ = "px-4 md:px-6 divide-y divide-customGray divide-opacity-25"):
        if item.text is not None:
            profile_dictionary['college'] = item.text
        else:
            profile_dictionary['college'] = 0
    for item in soup.find_all("div",class_ = "flex flex-col px-4 py-5 md:p-6 space-y-10"):
        if item.text is not None:
            profile_dictionary['test_score'] = item.text
        else:
            profile_dictionary['test_score'] = 0  
    for item in soup.find_all("div",class_ = "py-4 sm:py-7 first:sm:pt-4 last:sm:pb-4"):
        if item is not None:
            profile_dictionary['work_exp'] = item.text
        else:
            profile_dictionary['work_exp'] = 0
    all_profiles.append(profile_dictionary)
df_profiles = pd.DataFrame(all_profiles)

In [4]:
df_profiles['gre']=df_profiles['test_score'].apply(lambda x: str(x).partition('GRE')[2].replace('\n','').replace(' ','')[:3])
df_profiles['gre_quant']=df_profiles['test_score'].apply(lambda x: str(x).partition('Quant')[2].replace('\n','').replace(' ','')[:3])
df_profiles['gre_quant']=df_profiles['gre_quant'].apply(lambda x: '' if 'V' in x else x)
df_profiles['gre_verbal']=df_profiles['test_score'].apply(lambda x: str(x).partition('Verbal')[2].replace('\n','').replace(' ','')[:3])
df_profiles['gre_verbal']=df_profiles['gre_verbal'].apply(lambda x: '' if 'A' in x else x)

In [5]:
df_profiles['job_role']=df_profiles['work_exp'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('\n \n')[0].replace('\n','').strip())
df_profiles['work_company'] = df_profiles['work_exp'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('•')[0].partition('\n \n')[2].strip())
df_profiles['work_start_date'] = df_profiles['work_exp'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('•')[2].partition('\n \n')[2].split('-')[0].split('•')[0].strip())

In [6]:
df_profiles['cgpa_perc'] = df_profiles['college'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('\n \n')[0].strip())
df_profiles['cgpa_perc'] = df_profiles['cgpa_perc'].apply(lambda x: int(x[:1]) if '%' in x else x)
df_profiles['college_new'] = df_profiles['college'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('\n \n')[2].partition('\n \n')[2].partition('\n \n')[0].strip())

In [9]:
df_profiles['work_start_date'] = df_profiles['work_exp'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('•')[2].partition('\n \n')[2].split('-')[0].split('•')[0].strip())
end_date = datetime.datetime(int(years_list[1]),7,1) ## Assumption people leave jobs by July
df_profiles['work_start'] = pd.to_datetime(df_profiles['work_start_date'])
df_profiles['work_exp_months'] = (end_date.year - df_profiles['work_start'].dt.year) * 12 + (end_date.month - df_profiles['work_start'].dt.month)

## Yocket Student Applications

In [ ]:
for i in tqdm(id_list):
    driver.get("https://yocket.com/profile/{}#tab-applications".format(i[1:]))
    time.sleep(15)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    application_dictionary = {}
    application_dictionary['id'] = i
    for item in soup.find_all("div",class_="px-4 lg:px-0 py-1 lg:py-0"):
        application_dictionary['applications'] = item.text
    all_applications.append(application_dictionary)
df_applications = pd.DataFrame(all_applications)

In [13]:
application_course = []
application_id = []
for i in tqdm(range(0, df_applications.shape[0])):
    profile_id = df_applications['id'][i]
    school_list = re.split(r"\n \n|Premium Admit", re.sub(' +', ' ', df_applications['applications'][i]))
    profile_id_list = [profile_id] * len(school_list)
    application_course.extend(school_list)
    application_id.extend(profile_id_list)

100%|████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 7528.07it/s]


In [14]:
df_profile_applications = pd.DataFrame(
    {'id': application_id,
     'application_course': application_course
    })

In [15]:
df_profile_applications['school'] = df_profile_applications['application_course'].str.strip().str.partition('\n')[0]
df_profile_applications['course'] = df_profile_applications['application_course'].str.strip().str.partition('\n')[2].str.partition('\n')[2].str.strip().str.partition('\n')[0]
df_profile_applications['shortlist_status_temp'] = df_profile_applications['application_course'].apply(lambda x: str(x).strip().partition('\n')[2].partition('\n')[2].strip().partition('\n')[2].strip())
df_profile_applications['applied_date'] = df_profile_applications['shortlist_status_temp'].apply(lambda x: str(x).strip().partition('\n')[2].partition('\n')[0].strip())
df_profile_applications['univ_respond_date'] = df_profile_applications['shortlist_status_temp'].apply(lambda x: str(x).partition('University responded on:')[2].partition('\n')[2].partition('\n')[0].strip())
df_profile_applications['shortlist_status_actual'] = df_profile_applications['shortlist_status_temp'].apply(lambda x: 'Applied' if 'Applied' in str(x) else str(x))
df_profile_applications= df_profile_applications[df_profile_applications['school'] != 'View Comment']
df_profile_applications['shortlist_status_actual'] = df_profile_applications['shortlist_status_actual'].apply(lambda x: 'Shortlist' if len(x)>9  else str(x))

## Join the dataframes

In [17]:
df_final = df_profiles.merge(df_profile_applications, how='left',on='id')

In [19]:
df_final.drop(columns=['id','test_score','work_exp','work_start_date','application_course','shortlist_status_temp','college'],inplace=True)

## 2023 Data

In [95]:
application_course = []
application_id = []
for i in tqdm(range(0, df_applications.shape[0])):
    profile_id = df_applications['id'][i]
    school_list = re.split(r"\n \n|Premium Admit", re.sub(' +', ' ', df_applications['applications'][i]))
    profile_id_list = [profile_id] * len(school_list)
    application_course.extend(school_list)
    application_id.extend(profile_id_list)

100%|████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 1878.27it/s]


In [96]:
df_profile_applications = pd.DataFrame(
    {'id': application_id,
     'application_course': application_course
    })

In [97]:
df_profile_applications['school'] = df_profile_applications['application_course'].str.strip().str.partition('\n')[0]
df_profile_applications['course'] = df_profile_applications['application_course'].str.strip().str.partition('\n')[2].str.partition('\n')[2].str.strip().str.partition('\n')[0]
df_profile_applications['shortlist_status_temp'] = df_profile_applications['application_course'].apply(lambda x: str(x).strip().partition('\n')[2].partition('\n')[2].strip().partition('\n')[2].strip())
df_profile_applications['applied_date'] = df_profile_applications['shortlist_status_temp'].apply(lambda x: str(x).strip().partition('\n')[2].partition('\n')[0].strip())
df_profile_applications['univ_respond_date'] = df_profile_applications['shortlist_status_temp'].apply(lambda x: str(x).partition('University responded on:')[2].partition('\n')[2].partition('\n')[0].strip())
df_profile_applications['shortlist_status_actual'] = df_profile_applications['shortlist_status_temp'].apply(lambda x: 'Applied' if 'Applied' in str(x) else str(x))
df_profile_applications=df_profile_applications[df_profile_applications['school'] != 'View Comment']
df_profile_applications['shortlist_status_actual'] = df_profile_applications['shortlist_status_actual'].apply(lambda x: 'Shortlist' if len(x)>9  else str(x))

In [100]:
df_profiles['gre']=df_profiles['test_score'].apply(lambda x: str(x).partition('GRE')[2].replace('\n','').replace(' ','')[:3])
df_profiles['gre_quant']=df_profiles['test_score'].apply(lambda x: str(x).partition('Quant')[2].replace('\n','').replace(' ','')[:3])
df_profiles['gre_quant']=df_profiles['gre_quant'].apply(lambda x: '' if 'V' in x else x)
df_profiles['gre_verbal']=df_profiles['test_score'].apply(lambda x: str(x).partition('Verbal')[2].replace('\n','').replace(' ','')[:3])
df_profiles['gre_verbal']=df_profiles['gre_verbal'].apply(lambda x: '' if 'A' in x else x)
df_profiles['job_role']=df_profiles['work_exp'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('\n \n')[0].replace('\n','').strip())
df_profiles['work_company'] = df_profiles['work_exp'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('•')[0].partition('\n \n')[2].strip())
df_profiles['work_start_date'] = df_profiles['work_exp'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('•')[2].partition('\n \n')[2].split('-')[0].split('•')[0].strip())
df_profiles['cgpa_perc'] = df_profiles['college'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('\n \n')[0].strip())
df_profiles['cgpa_perc'] = df_profiles['cgpa_perc'].apply(lambda x: int(x[:1]) if '%' in x else x)
df_profiles['college_new'] = df_profiles['college'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('\n \n')[2].partition('\n \n')[2].partition('\n \n')[0].strip())
df_profiles['work_start_date'] = df_profiles['work_exp'].apply(lambda x: re.sub(' +', ' ', str(x)).partition('•')[2].partition('\n \n')[2].split('-')[0].split('•')[0].strip())
end_date = datetime.datetime(int(years_list[0]),7,1) ## Assumption people leave jobs by July
df_profiles['work_start'] = pd.to_datetime(df_profiles['work_start_date'])
df_profiles['work_exp_months'] = (end_date.year - df_profiles['work_start'].dt.year) * 12 + (end_date.month - df_profiles['work_start'].dt.month)


In [102]:
df_final = df_profiles.merge(df_profile_applications, how='left',on='id')

In [104]:
df_final.drop(columns=['test_score','work_exp','work_start_date','application_course','shortlist_status_temp','college'],inplace=True)

In [105]:
df_final['gre'] = pd.to_numeric(df_final['gre'])
df_final['gre_quant'] = pd.to_numeric(df_final['gre_quant'])
df_final['gre_verbal'] = pd.to_numeric(df_final['gre_verbal'])
#df_final['cgpa_perc'] = pd.to_numeric(df_final['cgpa_perc'])
#df_final['applied_date'] = pd.to_datetime(df_final['applied_date'])
df_final['univ_respond_date'] = pd.to_datetime(df_final['univ_respond_date'])

In [106]:
df_final.head(5)

,id,gre,gre_quant,gre_verbal,job_role,work_company,cgpa_perc,college_new,work_start,work_exp_months,school,course,applied_date,univ_respond_date,shortlist_status_actual
0,@Akhil_kumar Reddy_o1pwe,300.0,157.0,143.0,nan,,7,saividya institute of technology,NaT,NaN,University of New Haven,Data Science,,NaT,Shortlist
1,@Akhil_kumar Reddy_o1pwe,300.0,157.0,143.0,nan,,7,saividya institute of technology,NaT,NaN,University of Massachusetts Dartmouth,Data Science,,NaT,Shortlist
2,@Akhil_kumar Reddy_o1pwe,300.0,157.0,143.0,nan,,7,saividya institute of technology,NaT,NaN,Embry Riddle Aeronautical University,Data Science,,NaT,Shortlist
3,@Akhil_kumar Reddy_o1pwe,300.0,157.0,143.0,nan,,7,saividya institute of technology,NaT,NaN,"California State Polytechnic University, Pomona",Computer Science,,NaT,Shortlist
4,@Akhil_kumar Reddy_o1pwe,300.0,157.0,143.0,nan,,7,saividya institute of technology,NaT,NaN,"University of Massachusetts, Boston",Computer Science,,NaT,Shortlist


In [110]:
len(df_final['id'].value_counts())

45

## Analysis

In [166]:
req_list = df[(df['school']=='University of California, Davis') & (df['course'].str.contains('Business Analytics'))]['id'].to_list()

In [167]:
pd.set_option('display.max_rows',500)
df_1 = df[df['id'].isin(req_list)].groupby(['school','course']).size().reset_index()

In [168]:
df[df['id'].isin(req_list)].groupby(['school','course']).size().sort_values(ascending=False).reset_index()

,school,course,0
0,"University of California, Davis",Business Analytics,42
1,"University of California, San Diego",Business Analytics,28
2,University of Southern California,Business Analytics,13
3,Purdue University West Lafayette,Business Analytics and Information Management,13
4,University of Illinois at Chicago,Business Analytics,12
5,Arizona State University,Business Analytics,12
6,University of Cincinnati,Business Analytics,11
7,Michigan State University,Business Data Science and Analytics,11
8,"University of California, Los Angeles",Business Analytics,11
9,"University of California, Irvine",Business Analytics,10


In [158]:
df_1[df_1['school']=='University at Buffalo SUNY']

,school,course,0
208,University at Buffalo SUNY,Business Analytics,12
209,University at Buffalo SUNY,Computer Science and Engineering,1
210,University at Buffalo SUNY,Data Sciences and Applications,6
211,University at Buffalo SUNY,Engineering Science (Data Science),4
212,University at Buffalo SUNY,Management Information Systems,1
